# `train_ratings.csv` 로드해서 `MostPopular`  추천 결과 생성하기

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from models import BERTModel
from options import args
import torch
from utils import *

/tmp/ipykernel_207289/3339677482.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('datasets/data/train/train_ratings.csv') # 전체 학습 데이터
sub = pd.read_csv('datasets/data/eval/sample_submission.csv')


In [3]:
item_ids = df['item'].unique()
user_ids = df['user'].unique()
num_item, num_user = len(item_ids), len(user_ids)
num_batch = num_user // args.train_batch_size

In [4]:
item2idx = pd.Series(data=np.arange(len(item_ids))+1, index=item_ids)
user2idx = pd.Series(data=np.arange(len(user_ids)), index=user_ids)


In [5]:
df = pd.merge(df,pd.DataFrame({'item':item_ids, 'item_idx':item2idx[item_ids].values}),on='item',how='inner')
df = pd.merge(df,pd.DataFrame({'user':user_ids, 'user_idx':user2idx[user_ids].values}),on='user',how='inner')


In [6]:
df

,user,item,time,item_idx,user_idx
0,11,4643,1230782529,1,0
1,11,170,1230782534,2,0
2,11,531,1230782539,3,0
3,11,616,1230782542,4,0
4,11,2140,1230782563,5,0
...,...,...,...,...,...
5154466,138493,44022,1260209449,424,31359
5154467,138493,4958,1260209482,1492,31359
5154468,138493,68319,1260209720,332,31359
5154469,138493,40819,1260209726,734,31359


In [7]:
del df['item'], df['user']

df

,time,item_idx,user_idx
0,1230782529,1,0
1,1230782534,2,0
2,1230782539,3,0
3,1230782542,4,0
4,1230782563,5,0
...,...,...,...
5154466,1260209449,424,31359
5154467,1260209482,1492,31359
5154468,1260209720,332,31359
5154469,1260209726,734,31359


In [8]:
# Instantiate the BERT model
#model = BERTModel(args).to(args.device)

# Load the state dictionary
checkpoint_path = '/opt/ml/input/code/BERT4Rec-VAE-Pytorch/experiments/test_2024-02-02_18/models/best_acc_model.pth'
checkpoint = torch.load(checkpoint_path)
best_model_state_dict = checkpoint.get('model_state_dict')

# Load the state dictionary into the model
#model.load_state_dict(best_model_state_dict)

# Set the model to evaluation mode
#model.eval()


In [9]:
print(torch.__version__)
print(np.__version__)

1.11.0+cu102
1.26.3


In [10]:
df

,time,item_idx,user_idx
0,1230782529,1,0
1,1230782534,2,0
2,1230782539,3,0
3,1230782542,4,0
4,1230782563,5,0
...,...,...,...
5154466,1260209449,424,31359
5154467,1260209482,1492,31359
5154468,1260209720,332,31359
5154469,1260209726,734,31359


In [11]:
args.template = 'train_bert'
args.dataset_code='ml-2m'
args.train_negative_sampling_seed=args.test_negative_sampling_seed=0


In [12]:
print(args)


Namespace(mode='train', template='train_bert', test_model_path=None, dataset_code='ml-2m', min_rating=4, min_uc=5, min_sc=0, split='leave_one_out', dataset_split_seed=98765, eval_set_size=500, dataloader_code='bert', dataloader_random_seed=0.0, train_batch_size=64, val_batch_size=64, test_batch_size=64, train_negative_sampler_code='random', train_negative_sample_size=100, train_negative_sampling_seed=0, test_negative_sampler_code='random', test_negative_sample_size=100, test_negative_sampling_seed=0, trainer_code='bert', device='cpu', num_gpu=1, device_idx='0', optimizer='Adam', lr=0.001, weight_decay=0, momentum=None, decay_step=15, gamma=0.1, num_epochs=100, log_period_as_iter=12800, metric_ks=[10, 20, 50], best_metric='NDCG@10', find_best_beta=True, total_anneal_steps=2000, anneal_cap=0.2, model_code='bert', model_init_seed=None, bert_max_len=None, bert_num_items=None, bert_hidden_units=None, bert_num_blocks=None, bert_num_heads=None, bert_dropout=None, bert_mask_prob=None, dae_num_

In [15]:
print(args)

Namespace(mode='train', template='train_bert', test_model_path=None, dataset_code='ml-2m', min_rating=4, min_uc=5, min_sc=0, split='leave_one_out', dataset_split_seed=98765, eval_set_size=500, dataloader_code='bert', dataloader_random_seed=0.0, train_batch_size=64, val_batch_size=64, test_batch_size=64, train_negative_sampler_code='random', train_negative_sample_size=100, train_negative_sampling_seed=0, test_negative_sampler_code='random', test_negative_sample_size=100, test_negative_sampling_seed=0, trainer_code='bert', device='cpu', num_gpu=1, device_idx='0', optimizer='Adam', lr=0.001, weight_decay=0, momentum=None, decay_step=15, gamma=0.1, num_epochs=100, log_period_as_iter=12800, metric_ks=[10, 20, 50], best_metric='NDCG@10', find_best_beta=True, total_anneal_steps=2000, anneal_cap=0.2, model_code='bert', model_init_seed=None, bert_max_len=None, bert_num_items=None, bert_hidden_units=None, bert_num_blocks=None, bert_num_heads=None, bert_dropout=None, bert_mask_prob=None, dae_num_

In [17]:
args.mode = 'train'

args.dataset_code = 'ml-' + input('Input 2 for ml-2m, 1 for ml-1m, 20 for ml-20m: ') + 'm'
args.min_rating = 0 if args.dataset_code == 'ml-1m' else 4
args.min_uc = 5
args.min_sc = 0
args.split = 'leave_one_out'

args.dataloader_code = 'bert'
batch = 128
args.train_batch_size = batch
args.val_batch_size = batch
args.test_batch_size = batch

args.train_negative_sampler_code = 'random'
args.train_negative_sample_size = 0
args.train_negative_sampling_seed = 0
args.test_negative_sampler_code = 'random'
args.test_negative_sample_size = 100
args.test_negative_sampling_seed = 98765

args.trainer_code = 'bert'
args.device = 'cuda'
args.num_gpu = 1
args.device_idx = '0'
args.optimizer = 'Adam'
args.lr = 0.001
args.enable_lr_schedule = True
args.decay_step = 25
args.gamma = 1.0
args.num_epochs = 100 if args.dataset_code == 'ml-1m' else 200
args.metric_ks = [1, 5, 10, 20, 50, 100]
args.best_metric = 'NDCG@10'

args.model_code = 'bert'
args.model_init_seed = 0

args.bert_dropout = 0.1
args.bert_hidden_units = 256
args.bert_mask_prob = 0.15
args.bert_max_len = 100
args.bert_num_blocks = 2
args.bert_num_heads = 4

In [18]:
#Instantiate the BERT model
model = BERTModel(args).to('cuda')

#Load the state dictionary into the model
model.load_state_dict(best_model_state_dict)

#Set the model to evaluation mode
model.eval()

BERTModel(
  (bert): BERT(
    (embedding): BERTEmbedding(
      (token): TokenEmbedding(13528, 256, padding_idx=0)
      (position): PositionalEmbedding(
        (pe): Embedding(100, 256)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer_blocks): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadedAttention(
          (linear_layers): ModuleList(
            (0): Linear(in_features=256, out_features=256, bias=True)
            (1): Linear(in_features=256, out_features=256, bias=True)
            (2): Linear(in_features=256, out_features=256, bias=True)
          )
          (output_linear): Linear(in_features=256, out_features=256, bias=True)
          (attention): Attention()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=256, out_features=1024, bias=True)
          (w_2): Linear(in_features=1024, out_features=256, bias=True)
          (d

In [23]:
from dataloaders import bert
from datasets import dataset_factory

dataset = dataset_factory(args)
dataloader = bert.BertDataloader(args, dataset)
train, val, test = dataloader.get_pytorch_dataloaders()

Already preprocessed. Skip preprocessing
Negatives samples exist. Loading.
Negatives samples exist. Loading.


In [22]:
users = sub['user'].unique()
users

array([    11,     14,     18, ..., 138486, 138492, 138493])

## 모든 사용자에게 동일하게 Most Popular 아이템 10개를 추천 결과로 제공

In [ ]:
# most popular 10 items
num_users = train_df['user'].nunique()
mp_items = train_df['item'].value_counts()[:10].index.values # predictions
mp_items = np.concatenate([mp_items]*num_users) # same prediction for all users
users = train_df['user'].unique().repeat(10)

test_df = pd.DataFrame(zip(users,mp_items), columns=['user','item'])
test_df.to_csv("output/most_popular_submission.csv", index=False)

In [ ]:
test_df

,user,item
0,11,2571
1,11,2959
2,11,296
3,11,318
4,11,356
...,...,...
313595,138493,4993
313596,138493,7153
313597,138493,5952
313598,138493,593
